<a href="https://colab.research.google.com/github/SarkarPriyanshu/DataScienceAssign/blob/main/ResoluteAI_Task_3_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [74]:
import pandas as pd
import numpy as np
from functools import reduce

In [7]:
df = pd.read_excel('/content/gdrive/MyDrive/DataSets/Task3/rawdata.xlsx',sheet_name='inputsheet',parse_dates=['date'])

In [18]:
# Convert the column to datetime type
df['time_column'] = pd.to_datetime(df['time'], format='%H:%M:%S').dt.time

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         175 non-null    datetime64[ns]
 1   time         175 non-null    object        
 2   sensor       174 non-null    float64       
 3   location     175 non-null    object        
 4   number       175 non-null    int64         
 5   activity     175 non-null    object        
 6   position     175 non-null    object        
 7   location.1   175 non-null    object        
 8   time_column  175 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(6)
memory usage: 12.4+ KB


In [20]:
df.sample(5)

,date,time,sensor,location,number,activity,position,location.1,time_column
43,2024-01-16,00:42:20,0.0,A16,1,placed,inside,A16,00:42:20
13,2024-01-16,11:48:20,1.0,A14,1,picked,inside,A14,11:48:20
63,2024-01-16,01:12:20,1.0,A8,1,placed,inside,A8,01:12:20
85,2024-01-17,10:52:20,0.0,A2,1,placed,inside,A2,10:52:20
111,2024-01-18,12:35:30,1.0,A28,1,placed,inside,A28,12:35:30


In [26]:
df['activity'].value_counts()

placed    88
picked    87
Name: activity, dtype: int64

In [27]:
df['position'].value_counts()

inside     156
Inside      15
outside      4
Name: position, dtype: int64

In [81]:
def GetSummary(df=None):
    """
    GetSummary function aggregates data from the input DataFrame based on 'date',
    calculating counts of 'picked' and 'placed' activities, and the total duration
    spent 'inside' and 'outside' for each date.

    Parameters:
    - df (DataFrame): Input DataFrame containing columns 'date', 'activity',
      'position', and 'time_column'.

    Returns:
    - merged_df (DataFrame): Merged DataFrame containing the summary information
      aggregated based on 'date'.
    """

    # Calculate counts of 'picked' activities for each date
    picked_group = (df
                    .query("`activity` == 'picked'")
                    .groupby(['date'], as_index=False)
                    .agg({'activity':'count'}))

    # Calculate counts of 'placed' activities for each date
    placed_group = (df
                    .query("`activity` == 'placed'")
                    .groupby(['date'], as_index=False)
                    .agg({'activity':'count'}))

    # Calculate total duration spent 'inside' for each date
    inside_duration_group = (df
                             .query("`position` == 'inside'")
                             .groupby(['date'], as_index=False)
                             .agg({'time_column':lambda x: pd.to_timedelta(x.astype(str)).sum()}))

    # Calculate total duration spent 'outside' for each date
    outside_duration_group = (df
                              .query("`position` == 'outside'")
                              .groupby(['date'], as_index=False)
                              .agg({'time_column':lambda x: pd.to_timedelta(x.astype(str)).sum()}))

    # Rename columns for clarity
    picked_group = picked_group.rename(columns={'activity':'pick_activities'})
    placed_group = placed_group.rename(columns={'activity':'placed_activities'})
    inside_duration_group = inside_duration_group.rename(columns={'time_column':'inside_duration'})
    outside_duration_group = outside_duration_group.rename(columns={'time_column':'outside_duration'})

    # Merge the calculated summaries based on 'date'
    dfs = [picked_group, placed_group, inside_duration_group, outside_duration_group]
    merged_df = reduce(lambda left, right: pd.merge(left, right, on='date', how='outer'), dfs)

    return merged_df


In [82]:
GetSummary(df)

,date,pick_activities,placed_activities,inside_duration,outside_duration
0,2024-01-16,40,40,9 days 06:02:20,NaT
1,2024-01-17,10,9,9 days 00:47:50,NaT
2,2024-01-18,37,39,35 days 06:57:29,2 days 15:31:30
